# Dependence of spike widths and amplitudes on distance from soma

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import LFPy
import neuron
from neuron import h
import brainsignals.neural_simulations as ns
from brainsignals.plotting_convention import mark_subplots, simplify_axes
from brainsignals.neural_simulations import return_hay_cell

ns.load_mechs_from_folder(ns.cell_models_folder)
np.random.seed(12345)

tstop = 150
dt = 2**-6

# Time window to extract spike from:
t0 = 118
t1 = 123

In [ ]:
def insert_current_stimuli(cell):
    stim_params = {'amp': -0.4,
                   'idx': 0,
                   'pptype': "ISyn",
                   'dur': 1e9,
                   'delay': 0}
    synapse = LFPy.StimIntElectrode(cell, **stim_params)
    return synapse, cell

In [ ]:
def return_electrode_params():

    x_elec = np.linspace(10, 200, 50)
    num_elecs = len(x_elec)
    elec_grid_params = dict(
                sigma = 0.3,      # extracellular conductivity
                x = x_elec,
                y = np.zeros(num_elecs),
                z = np.zeros(num_elecs),
                method = 'root_as_point',
            )
    return elec_grid_params

def get_spike_width(eaps, tvec, name='test', plot_eaps=False):
    # Returns the spike width at 25 percent of negative maximum value
    widths = []

    for i, eap in enumerate(eaps):
        neg_peak_25percent = np.min(eap) / 4
        crossings_down = ((eap[:-1] > neg_peak_25percent) &
                     (eap[1:] <= neg_peak_25percent))
        crossings_up = ((eap[:-1] < neg_peak_25percent) &
                     (eap[1:] >= neg_peak_25percent))

        t0_idx = np.where(crossings_down)[0][0]

        if len(np.where(crossings_up)[0]) == 0:
            # If EAP doesn't go above threshold again
            t1_idx = -1
        else:
            t1_idx = np.where(crossings_up)[0][-1]

        width = tvec[t1_idx] - tvec[t0_idx]

        if plot_eaps:
            # Code to debug spike width function, or investiage spike shapes. 
            # Spike shapes often start looking really wierd after a certain distance, 
            # and 'spike width' is hardly a well-defined concept anymore.
            figfolder = "eap_widths"
            os.makedirs(figfolder, exist_ok=True)
            plt.close("all")
            plt.plot(tvec, eap)
            plt.axhline(neg_peak_25percent, ls=':', c='gray', lw=0.7)
            plt.axvline(tvec[t0_idx], ls=':', c='gray', lw=0.7)
            plt.plot([tvec[t0_idx], tvec[t0_idx] + width],
                     [neg_peak_25percent, neg_peak_25percent], c='k', lw=3)
            plt.text(tvec[t0_idx] + width / 2, neg_peak_25percent, "%f ms" % width,
                     va="bottom")
            plt.axvline(tvec[t1_idx], ls=':', c='gray', lw=0.7)
            plt.savefig(os.path.join(figfolder, "eap_width_%s_%i.png" % (name, i)))

        widths.append(width)

    return widths

## First we run original simulation with active conductances.  The somatic membrane potential from this simulation will be replayed into simpler models 

In [ ]:
cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=False)

print(cell.d[cell.somaidx])

ns.point_axon_down(cell)
syn, cell = insert_current_stimuli(cell)
cell.simulate(rec_imem=True, rec_vmem=True)
t0_idx = np.argmin(np.abs(cell.tvec - t0))
t1_idx = np.argmin(np.abs(cell.tvec - t1))

cell.vmem = cell.vmem[:, t0_idx:t1_idx]
cell.imem = cell.imem[:, t0_idx:t1_idx]
cell.tvec = cell.tvec[t0_idx:t1_idx] - cell.tvec[t0_idx]

np.save("somatic_vmem.npy", [cell.tvec, cell.vmem[0, :]])
np.save("imem_orig.npy", cell.imem)
np.save("vmem_orig.npy", cell.vmem)

elec_params = return_electrode_params()

elec = LFPy.RecExtElectrode(cell, **elec_params)
M_elec = elec.get_transformation_matrix()
eaps = M_elec @ cell.imem * 1000

widths_hay_active = get_spike_width(eaps, cell.tvec, "active_hay", False)
p2p_hay_active = np.max(eaps, axis=1) - np.min(eaps, axis=-1)

cell.__del__()

## Then we replay the simulated somatic membrane potential into the somas of different models. We start with the passive hay-model

In [ ]:
soma_t, soma_vmem = np.load("somatic_vmem.npy")

cell = return_hay_cell(tstop=soma_t[-1], dt=dt, make_passive=False)
ns.point_axon_down(cell)
remove_list = ["Nap_Et2", "NaTa_t", "NaTs2_t", "SKv3_1",
               "SK_E2", "K_Tst", "K_Pst",
               "Im", "Ih", "CaDynamics_E2", "Ca_LVAst", "Ca", "Ca_HVA"]
cell = ns.remove_active_mechanisms(remove_list, cell)
h.dt = dt

for sec in neuron.h.allsec():
    if "soma" in sec.name():
        print("g_pas: {}, e_pas: {}, cm: {}, "
              "Ra: {}, soma_diam: {}, soma_L: {}".format(sec.g_pas,
                                                         sec.e_pas, sec.cm,
                                                         sec.Ra, sec.diam,
                                                         sec.L))
        vclamp = h.SEClamp_i(sec(0.5))
        vclamp.dur1 = 1e9
        vclamp.rs = 1e-9
        vmem_to_insert = h.Vector(soma_vmem[1:])
        vmem_to_insert.play(vclamp._ref_amp1, h.dt)

cell.simulate(rec_imem=True, rec_vmem=True)

elec_params = return_electrode_params()
elec = LFPy.RecExtElectrode(cell, **elec_params)
M_elec = elec.get_transformation_matrix()
eaps = M_elec @ cell.imem * 1000

widths_hay_replay = get_spike_width(eaps, cell.tvec, "hay_replay",False)
p2p_hay_replay = np.max(eaps, axis=1) - np.min(eaps, axis=-1)

cell.__del__()

## Now we try a ball and stick neuron

In [ ]:
cell = ns.return_ball_and_stick_cell(soma_t[-1], dt, apic_diam=4)
for sec in neuron.h.allsec():
    # Insert same passive params as Hay model
    sec.g_pas = 3.38e-05
    sec.e_pas = -90
    sec.cm = 1.0
    sec.Ra = 100
h.dt = dt

for sec in neuron.h.allsec():
    if "soma" in sec.name():
        vclamp = h.SEClamp_i(sec(0.5))
        vclamp.dur1 = 1e9
        vclamp.rs = 1e-9
        vmem_to_insert = h.Vector(soma_vmem)
        vmem_to_insert.play(vclamp._ref_amp1, h.dt)

cell.simulate(rec_imem=True, rec_vmem=True)

elec_params = return_electrode_params()
elec = LFPy.RecExtElectrode(cell, **elec_params)
M_elec = elec.get_transformation_matrix()
eaps = M_elec @ cell.imem * 1000

widths_bns = get_spike_width(eaps, cell.tvec, "ball_and_stick", False)
p2p_bns = np.max(eaps, axis=1) - np.min(eaps, axis=-1)

cell.__del__()

## Plot results

In [ ]:
fig = plt.figure(figsize=[6, 2.2])
fig.subplots_adjust(bottom=0.23, left=0.07, right=0.98, top=0.9)
ax_w = fig.add_subplot(121, ylabel="spike width (ms)", 
                       xlabel="distance (µm)", 
                       xticks=[10, 50, 100, 150, 200], 
                       xlim=[10, 200], ylim=[0.3, 1.4])
ax_p2p = fig.add_subplot(122, ylabel="amplitude (µV)",
                         xlabel="distance (µm)")

l1, = ax_w.plot(elec.x, widths_hay_replay, 'k')
ax_p2p.loglog(elec.x, p2p_hay_replay, 'k')

l2, = ax_w.plot(elec.x, widths_bns, 'gray')
ax_p2p.loglog(elec.x, p2p_bns, 'gray')

# Plot powerlaw behaviour
r_end = elec.x[np.where(elec.x > 100)] 
r_start = elec.x[np.where(elec.x < 20)] 

p_clr = ['darkred', 'red', 'tomato']
for pow_idx, r_pow in enumerate([1, 2, 3]):
    for v in [p2p_bns[-1]]:
        K = v * (r_end[-1]**r_pow)
        y = K / (r_end ** r_pow)
        ax_p2p.loglog(r_end, y, '-', c=p_clr[pow_idx])
        
    for v in [p2p_bns[0]]:
        K = v * (r_start[0]**r_pow)
        y = K / (r_start ** r_pow)
        ax_p2p.loglog(r_start, y, '-', c=p_clr[pow_idx])        
    
    ax_p2p.text(10, [1, 2, 4][pow_idx], 
                "r$^{-%d}$" % r_pow, color=p_clr[pow_idx],
               fontsize=14)
       
mark_subplots(fig.axes, ypos=1.07)
simplify_axes(fig.axes)
fig.legend([l1 ,l2], ["Hay neuron model", "ball and stick"],
          frameon=False, loc=(0.25, 0.0), ncol=2)

fig.savefig("width_amp_hay_bns.pdf")

In [ ]:
elec_idx_20um = np.argmin(np.abs(elec.x - 20))
elec_idx_100um = np.argmin(np.abs(elec.x - 100))

print("Hay p2p amp at %1.1f µm: %1.2f µV" % (elec.x[elec_idx_20um], p2p_hay_replay[elec_idx_20um]))
print("Hay p2p amp at %1.1f µm: %1.2f µV" % (elec.x[elec_idx_100um], p2p_hay_replay[elec_idx_100um]))